In [165]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import fetch_california_housing, load_breast_cancer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from collections import defaultdict
from tqdm.notebook import tqdm
from itertools import product

import math
import graphviz
import numpy as np
import pandas as pd
import xgboost as xgb

### XGBoost from-scract регрессия

In [ ]:
X, y = fetch_california_housing(as_frame=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

In [ ]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
9694,2.1766,24.0,3.690883,1.052707,3243.0,4.619658,36.68,-121.63
18859,2.2462,52.0,5.692635,1.167139,1728.0,2.447592,41.32,-122.09
11074,2.8235,34.0,4.065760,1.088435,1722.0,3.904762,33.80,-117.86
10858,3.3750,36.0,5.162162,1.135135,208.0,5.621622,33.71,-117.86
16462,2.4482,11.0,4.150919,1.041995,2106.0,2.763780,38.11,-121.27


In [ ]:
y_train.head()

9694     1.085
18859    0.629
11074    1.531
10858    1.575
16462    1.030
Name: MedHouseVal, dtype: float64

In [ ]:
# def visualize_tree(tree, feature_names, file_name):
#     dot = graphviz.Digraph(comment='XGBoost Tree')
#     dot.node('root', tree.to_dot(feature_names))
#     dot.render(file_name, view=True)

In [ ]:
class MSE():
    def loss(self, y, pred): return np.mean((y - pred)**2)
    def gradient(self, y, pred): return pred - y
    def hessian(self, y, pred): return np.ones(len(y))

In [ ]:
class XGBoostRegressionBinarySplit():
    def __init__(self, params, random_seed=None):
        self.params = defaultdict(lambda: None, params)
        self.subsample = self.params['subsample'] \
            if self.params['subsample'] else 1.0
        self.learning_rate = self.params['learning_rate'] \
            if self.params['learning_rate'] else 0.3
        self.base_prediction = self.params['base_score'] \
            if self.params['base_score'] else 0.5
        self.max_depth = self.params['max_depth'] \
            if self.params['max_depth'] else 5
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y, objective, num_boost_round, verbose=False):
        current_predictions = self.base_prediction * np.ones(shape=y.shape)
        self.boosters = []
        for i in range(num_boost_round):
            gradients = objective.gradient(y, current_predictions)
            hessians = objective.hessian(y, current_predictions)
            sample_idxs = None if self.subsample == 1.0 else self.rng.choice(len(y), size=math.floor(self.subsample*len(y)), replace=False)
            booster = TreeBoosterRegressionBinary(X, gradients, hessians,self.params, self.max_depth, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)
            if verbose:
                print(f'[{i}] train loss = {objective.loss(y, current_predictions)}')

    def predict(self, X):
        return (self.base_prediction + self.learning_rate * np.sum([booster.predict(X) for booster in self.boosters], axis=0))

class TreeBoosterRegressionBinary():
    def __init__(self, X, g, h, params, max_depth, idxs=None):
        self.params = params
        self.max_depth = max_depth
        assert self.max_depth >= 0, 'max_depth must be nonnegative'
        self.min_child_weight = params['min_child_weight'] \
            if params['min_child_weight'] else 1.0
        self.reg_lambda = params['reg_lambda'] if params['reg_lambda'] else 1.0
        self.gamma = params['gamma'] if params['gamma'] else 0.0
        self.colsample_bynode = params['colsample_bynode'] \
            if params['colsample_bynode'] else 1.0
        if isinstance(g, pd.Series): g = g.values
        if isinstance(h, pd.Series): h = h.values
        if idxs is None: idxs = np.arange(len(g))
        self.X, self.g, self.h, self.idxs = X, g, h, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.reg_lambda) # Eq (5)
        self.best_score_so_far = 0.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)

        if self.is_leaf: return

        x = self.X.values[self.idxs,self.split_feature_idx]
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]

        self.left = TreeBoosterRegressionBinary(self.X, self.g, self.h, self.params,
                                self.max_depth - 1, self.idxs[left_idx])
        self.right = TreeBoosterRegressionBinary(self.X, self.g, self.h, self.params,
                                 self.max_depth - 1, self.idxs[right_idx])

    @property
    def is_leaf(self): return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X.values[self.idxs, feature_idx]
        g, h = self.g[self.idxs], self.h[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.

        for i in range(0, self.n - 1):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i; sum_g_right -= g_i
            sum_h_left += h_i; sum_h_right -= h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:continue
            if sum_h_right < self.min_child_weight: break

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                            + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                            - (sum_g**2 / (sum_h + self.reg_lambda))
                            ) - self.gamma/2 # Eq(7) in the xgboost paper

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold = (x_i + x_i_next) / 2

    def predict(self, X):
        return np.array([self._predict_row(row) for i, row in X.iterrows()])

    def _predict_row(self, row):
        if self.is_leaf:
            return self.value
        child = self.left if row[self.split_feature_idx] <= self.threshold else self.right

        return child._predict_row(row)

    # def to_dot(self, feature_names):
    #     if self.is_leaf:
    #         return f'leaf ({self.value:.2f})'

    #     left_dot = self.left.to_dot(feature_names) if self.left else 'leaf (0.0)'
    #     right_dot = self.right.to_dot(feature_names) if self.right else 'leaf (0.0)'

    #     split_feature_name = feature_names[self.split_feature_idx]
    #     condition = f'{split_feature_name} <= {self.threshold}'

    #     return f'node {{split: {split_feature_name}, thresholds: [{self.threshold}]\nleft: {left_dot}\nright: {right_dot}}}'

In [ ]:
class XGBoostRegressionTernarySplit():
    def __init__(self, params, random_seed=None):
        self.params = defaultdict(lambda: None, params)
        self.subsample = self.params['subsample'] \
            if self.params['subsample'] else 1.0
        self.learning_rate = self.params['learning_rate'] \
            if self.params['learning_rate'] else 0.3
        self.base_prediction = self.params['base_score'] \
            if self.params['base_score'] else 0.5
        self.max_depth = self.params['max_depth'] \
            if self.params['max_depth'] else 5
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y, objective, num_boost_round, verbose=False):
        current_predictions = self.base_prediction * np.ones(shape=y.shape)
        self.boosters = []
        for i in range(num_boost_round):
            gradients = objective.gradient(y, current_predictions)
            hessians = objective.hessian(y, current_predictions)
            sample_idxs = None if self.subsample == 1.0 else self.rng.choice(len(y), size=math.floor(self.subsample*len(y)), replace=False)
            booster = TreeBoosterRegressionTernary(X, gradients, hessians, self.params, self.max_depth, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)
            if verbose:
                print(f'[{i}] train loss = {objective.loss(y, current_predictions)}')

    def predict(self, X):
        return (self.base_prediction + self.learning_rate * np.sum([booster.predict(X) for booster in self.boosters], axis=0))

class TreeBoosterRegressionTernary():
    def __init__(self, X, g, h, params, max_depth, idxs=None):
        self.left = None
        self.middle = None
        self.right = None
        self.split_feature_idx = None
        self.threshold1 = None
        self.threshold2 = None

        self.params = params
        self.max_depth = max_depth
        assert self.max_depth >= 0, 'max_depth must be nonnegative'
        self.min_child_weight = params['min_child_weight'] \
            if params['min_child_weight'] else 1.0
        self.reg_lambda = params['reg_lambda'] if params['reg_lambda'] else 1.0
        self.gamma = params['gamma'] if params['gamma'] else 0.0
        self.colsample_bynode = params['colsample_bynode'] \
            if params['colsample_bynode'] else 1.0
        if isinstance(g, pd.Series): g = g.values
        if isinstance(h, pd.Series): h = h.values
        if idxs is None: idxs = np.arange(len(g))
        self.X, self.g, self.h, self.idxs = X, g, h, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.reg_lambda) # Eq (5)
        self.best_score_so_far = 0.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)

        if self.is_leaf: return

        x = self.X.values[self.idxs, self.split_feature_idx]
        left_idx = np.nonzero((x <= self.threshold1))[0]
        middle_idx = np.nonzero((x > self.threshold1) & (x <= self.threshold2))[0]
        right_idx = np.nonzero((x > self.threshold2))[0]

        self.left = TreeBoosterRegressionTernary(self.X, self.g, self.h, self.params,
                                self.max_depth - 1, self.idxs[left_idx])
        self.middle = TreeBoosterRegressionTernary(self.X, self.g, self.h, self.params,
                                  self.max_depth - 1, self.idxs[middle_idx])
        self.right = TreeBoosterRegressionTernary(self.X, self.g, self.h, self.params,
                                 self.max_depth - 1, self.idxs[right_idx])

    @property
    def is_leaf(self):
        return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X.values[self.idxs, feature_idx]
        g, h = self.g[self.idxs], self.h[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.
        sum_g_middle, sum_h_middle = 0., 0.

        for i in range(0, self.n - 2):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i; sum_g_right -= g_i; sum_g_middle += g_i
            sum_h_left += h_i; sum_h_right -= h_i; sum_h_middle += h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:
                continue
            if sum_h_right < self.min_child_weight:
                break

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                          + (sum_g_middle**2 / (sum_h_middle + self.reg_lambda))
                          + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                          - (sum_g**2 / (sum_h + self.reg_lambda))
                          ) - self.gamma

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold1 = (x_i + x_i_next) / 2
                self.threshold2 = (x_i_next + sort_x[i + 2]) / 2

    def predict(self, X):
        return np.array([self._predict_row(row) for i, row in X.iterrows()])

    def _predict_row(self, row):
        if self.is_leaf:
            return self.value
        if row[self.split_feature_idx] <= self.threshold1:
            child = self.left
        elif row[self.split_feature_idx] <= self.threshold2:
            child = self.middle
        else:
            child = self.right
        return child._predict_row(row)

    # def to_dot(self, feature_names):
    #     if self.is_leaf:
    #         return f'leaf ({self.value:.2f})'

    #     left_dot = self.left.to_dot(feature_names) if self.left else 'leaf (0.0)'
    #     middle_dot = self.middle.to_dot(feature_names) if self.middle else 'leaf (0.0)'
    #     right_dot = self.right.to_dot(feature_names) if self.right else 'leaf (0.0)'

    #     split_feature_name = feature_names[self.split_feature_idx]
    #     condition = f'{split_feature_name} <= {self.threshold1}'
    #     condition_middle = f'{self.threshold1} < {split_feature_name} <= {self.threshold2}'

    #     return f'node {{split: {split_feature_name}, thresholds: [{self.threshold1}, {self.threshold2}]\nleft: {left_dot}\nmiddle: {middle_dot}\nright: {right_dot}}}'

In [ ]:
def find_hyperparams(X_train, X_test, y_train, y_test):
    best_score = 100

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    learning_rates = [0.01, 0.05, 0.1]
    max_depths = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    reg_lambdas = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    gammas = [0.5, 0.75, 1.0, 1.5, 1.75, 2.0]
    min_child_weights = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

    combs = list(product(*[learning_rates, max_depths, reg_lambdas, gammas, min_child_weights]))

    for comb in tqdm(combs):
        learning_rate, max_depth, reg_lambda, gamma, min_child_weight = comb

        params = {
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'subsample': 1.0,
            'reg_lambda': reg_lambda,
            'gamma': gamma,
            'min_child_weight': min_child_weight,
            'base_score': 0.0,
            'device': 'cuda:0'
        }

        model_xgb = xgb.train(params, dtrain, num_boost_round=50)
        pred_xgb = model_xgb.predict(dtest)

        score = MSE().loss(y_test, pred_xgb)

        if score < best_score:
            best_score = score
            best_params = params

    return best_params

In [ ]:
best_params = find_hyperparams(X_train, X_test, y_train, y_test)

  0%|          | 0/12600 [00:00<?, ?it/s]

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

model_xgb = xgb.train(best_params, dtrain, num_boost_round=50)
pred_xgb = model_xgb.predict(dtest)

print(f'XGBoost Regression best params: {best_params.items()}')

print(f'Squared error xgboost score: {MSE().loss(y_test, pred_xgb)}')
print(f'MAPE xgboost score: {mean_absolute_percentage_error(y_test, pred_xgb)}')
print(f'R2 xgboost score: {r2_score(y_test, pred_xgb)}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:17:36] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "method" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Regression best params: dict_items([('learning_rate', 0.1), ('max_depth', 15), ('subsample', 1.0), ('reg_lambda', 3.0), ('gamma', 0.5), ('min_child_weight', 25), ('base_score', 0.0), ('method', 'exact')])
Squared error xgboost score: 0.1926893917699024
MAPE xgboost score: 0.16381667079845627
R2 xgboost score: 0.8511408361964348


In [ ]:
best_params = {
    'learning_rate': 0.1,
    'max_depth': 15,
    'subsample': 1.0,
    'reg_lambda': 3.0,
    'gamma': 0.5,
    'min_child_weight': 25,
    'base_score': 0.0,
    'method': 'exact'
}

In [ ]:
model_scratch_binary_split = XGBoostRegressionBinarySplit(best_params, random_seed=37)
model_scratch_binary_split.fit(X_train, y_train, MSE(), num_boost_round=50)

In [ ]:
model_scratch_ternary_split = XGBoostRegressionTernarySplit(best_params, random_seed=37)
model_scratch_ternary_split.fit(X_train, y_train, MSE(), num_boost_round=50)

In [ ]:
pred_scratch_binary = model_scratch_binary_split.predict(X_test)
pred_scratch_ternary = model_scratch_ternary_split.predict(X_test)

In [ ]:
print(f'MSE binary scratch score: {MSE().loss(y_test, pred_scratch_binary)}')
print(f'MSE error ternary scratch score: {MSE().loss(y_test, pred_scratch_ternary)}')

MSE binary scratch score: 0.1943774890704353
MSE error ternary scratch score: 0.18923895770318658


In [ ]:
print(f'MAPE binary scratch score: {mean_absolute_percentage_error(y_test, pred_scratch_binary)}')
print(f'MAPE ternary scratch score: {mean_absolute_percentage_error(y_test, pred_scratch_ternary)}')

MAPE binary scratch score: 0.165964509910908
MAPE ternary scratch score: 0.16505439906473393


In [ ]:
print(f'R2 binary scratch score: {r2_score(y_test, pred_scratch_binary)}')
print(f'R2 ternary scratch score: {r2_score(y_test, pred_scratch_ternary)}')

R2 binary scratch score: 0.8498367231351591
R2 ternary scratch score: 0.853806414852389


### XGBoost from-scratch классификация

In [116]:
X, y = load_breast_cancer(as_frame=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

In [117]:
sc = StandardScaler()

X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X.columns)

In [118]:
X_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,-0.547360,-0.315728,-0.577831,-0.568954,-0.711833,-0.716341,-0.625731,-0.660689,0.552566,-0.064394,...,-0.576510,-0.068422,-0.634201,-0.584220,-0.482419,-0.489727,-0.383476,-0.508605,0.305726,-0.144908
1,-0.909536,-0.490486,-0.846200,-0.822267,-0.527963,0.118327,0.060981,-0.334971,-1.190245,0.501507,...,-0.833920,0.206141,-0.527960,-0.744111,0.275291,0.963476,0.964894,0.608331,-0.598878,0.615108
2,1.167319,0.307735,1.167389,1.118944,0.715446,0.371483,0.829156,1.148293,0.154003,-0.458601,...,1.154209,0.094651,1.034442,1.107200,0.715111,-0.015884,0.479937,0.539844,0.244156,-0.290939
3,0.247731,0.130615,0.326171,0.130795,0.511147,0.943430,0.990605,0.986704,0.135886,0.012525,...,0.394534,-0.233160,0.473226,0.254934,0.972036,1.040127,1.166483,1.460605,-0.134736,-0.089535
4,-1.352353,0.572234,-1.343955,-1.108385,0.546371,-0.813853,-0.852252,-0.933561,0.751848,0.447939,...,-1.149928,0.933316,-1.170508,-0.957177,0.863169,-0.866647,-1.015298,-1.184489,-0.074744,-0.051534


In [119]:
y_train.head()

155    1
440    1
516    0
94     0
416    1
Name: target, dtype: int64

In [120]:
# def visualize_tree(tree, feature_names, file_name):
#     dot = graphviz.Digraph(comment='XGBoost Tree')
#     dot.node('root', tree.to_dot(feature_names))
#     dot.render(file_name, view=True)

In [121]:
def get_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 score:", f1)

In [122]:
class LogLoss():
    def loss(self, y, pred):
        preds = 1 / (1 + np.exp(-pred))
        return -np.mean(y * np.log(preds) + (1 - y) * np.log(1 - preds))

    def gradient(self, y, pred):
        preds = 1 / (1 + np.exp(-pred))
        return preds - y

    def hessian(self, y, pred):
        preds = 1 / (1 + np.exp(-pred))
        return preds * (1 - preds)

In [123]:
class XGBoostClassificationBinarySplit():
    def __init__(self, params, random_seed=None):
        self.params = defaultdict(lambda: None, params)
        self.subsample = self.params['subsample'] if self.params['subsample'] else 1.0
        self.learning_rate = self.params['learning_rate'] if self.params['learning_rate'] else 0.3
        self.base_prediction = self.params['base_score'] if self.params['base_score'] else 0.5
        self.max_depth = self.params['max_depth'] if self.params['max_depth'] else 5
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y, objective, num_boost_round, verbose=False):
        current_predictions = self.base_prediction * np.ones(shape=y.shape)
        self.boosters = []
        for i in range(num_boost_round):
            gradients = objective.gradient(y, current_predictions)
            hessians = objective.hessian(y, current_predictions)
            sample_idxs = None if self.subsample == 1.0 else self.rng.choice(len(y), size=math.floor(self.subsample*len(y)), replace=False)
            booster = TreeBoosterClassificationBinary(X, gradients, hessians, self.params, self.max_depth, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)
            if verbose:
                print(f'[{i}] train loss = {objective.loss(y, current_predictions)}')

    def predict(self, X):
        raw_preds = self.base_prediction + self.learning_rate * np.sum([booster.predict(X) for booster in self.boosters], axis=0)
        return 1 / (1 + np.exp(-raw_preds))

class TreeBoosterClassificationBinary():
    def __init__(self, X, g, h, params, max_depth, idxs=None):
        self.params = params
        self.max_depth = max_depth
        assert self.max_depth >= 0, 'max_depth must be nonnegative'
        self.min_child_weight = params['min_child_weight'] \
            if params['min_child_weight'] else 1.0
        self.reg_lambda = params['reg_lambda'] if params['reg_lambda'] else 1.0
        self.gamma = params['gamma'] if params['gamma'] else 0.0
        self.colsample_bynode = params['colsample_bynode'] \
            if params['colsample_bynode'] else 1.0
        if isinstance(g, pd.Series): g = g.values
        if isinstance(h, pd.Series): h = h.values
        if idxs is None: idxs = np.arange(len(g))
        self.X, self.g, self.h, self.idxs = X, g, h, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.reg_lambda) # Eq (5)
        self.best_score_so_far = 0.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)

        if self.is_leaf: return

        x = self.X.values[self.idxs,self.split_feature_idx]
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]

        self.left = TreeBoosterClassificationBinary(self.X, self.g, self.h, self.params,
                                self.max_depth - 1, self.idxs[left_idx])
        self.right = TreeBoosterClassificationBinary(self.X, self.g, self.h, self.params,
                                 self.max_depth - 1, self.idxs[right_idx])

    @property
    def is_leaf(self): return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X.values[self.idxs, feature_idx]
        g, h = self.g[self.idxs], self.h[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.

        for i in range(0, self.n - 1):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i; sum_g_right -= g_i
            sum_h_left += h_i; sum_h_right -= h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:continue
            if sum_h_right < self.min_child_weight: break

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                            + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                            - (sum_g**2 / (sum_h + self.reg_lambda))
                            ) - self.gamma/2 # Eq(7) in the xgboost paper

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold = (x_i + x_i_next) / 2

    def predict(self, X):
        return np.array([self._predict_row(row) for i, row in X.iterrows()])

    def _predict_row(self, row):
        if self.is_leaf:
            return self.value
        child = self.left if row[self.split_feature_idx] <= self.threshold else self.right

        return child._predict_row(row)

    # def to_dot(self, feature_names):
    #     if self.is_leaf:
    #         return f'leaf ({self.value:.2f})'

    #     left_dot = self.left.to_dot(feature_names) if self.left else 'leaf (0.0)'
    #     right_dot = self.right.to_dot(feature_names) if self.right else 'leaf (0.0)'

    #     split_feature_name = feature_names[self.split_feature_idx]
    #     condition = f'{split_feature_name} <= {self.threshold}'

    #     return f'node {{split: {split_feature_name}, thresholds: [{self.threshold}]\nleft: {left_dot}\nright: {right_dot}}}'

In [124]:
class XGBoostClassificationTernarySplit():
    def __init__(self, params, random_seed=None):
        self.params = defaultdict(lambda: None, params)
        self.subsample = self.params['subsample'] if self.params['subsample'] else 1.0
        self.learning_rate = self.params['learning_rate'] if self.params['learning_rate'] else 0.3
        self.base_prediction = self.params['base_score'] if self.params['base_score'] else 0.5
        self.max_depth = self.params['max_depth'] if self.params['max_depth'] else 5
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y, objective, num_boost_round, verbose=False):
        current_predictions = self.base_prediction * np.ones(shape=y.shape)
        self.boosters = []
        for i in range(num_boost_round):
            gradients = objective.gradient(y, current_predictions)
            hessians = objective.hessian(y, current_predictions)
            sample_idxs = None if self.subsample == 1.0 else self.rng.choice(len(y), size=math.floor(self.subsample*len(y)), replace=False)
            booster = TreeBoosterClassificationTernary(X, gradients, hessians, self.params, self.max_depth, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)
            if verbose:
                print(f'[{i}] train loss = {objective.loss(y, current_predictions)}')

    def predict(self, X):
        raw_preds = self.base_prediction + self.learning_rate * np.sum([booster.predict(X) for booster in self.boosters], axis=0)
        return 1 / (1 + np.exp(-raw_preds))

class TreeBoosterClassificationTernary():
    def __init__(self, X, g, h, params, max_depth, idxs=None):
        self.left = None
        self.middle = None
        self.right = None
        self.split_feature_idx = None
        self.threshold1 = None
        self.threshold2 = None

        self.params = params
        self.max_depth = max_depth
        assert self.max_depth >= 0, 'max_depth must be nonnegative'
        self.min_child_weight = params['min_child_weight'] \
            if params['min_child_weight'] else 1.0
        self.reg_lambda = params['reg_lambda'] if params['reg_lambda'] else 1.0
        self.gamma = params['gamma'] if params['gamma'] else 0.0
        self.colsample_bynode = params['colsample_bynode'] \
            if params['colsample_bynode'] else 1.0
        if isinstance(g, pd.Series): g = g.values
        if isinstance(h, pd.Series): h = h.values
        if idxs is None: idxs = np.arange(len(g))
        self.X, self.g, self.h, self.idxs = X, g, h, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.reg_lambda) # Eq (5)
        self.best_score_so_far = 0.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)

        if self.is_leaf: return

        x = self.X.values[self.idxs, self.split_feature_idx]
        left_idx = np.nonzero((x <= self.threshold1))[0]
        middle_idx = np.nonzero((x > self.threshold1) & (x <= self.threshold2))[0]
        right_idx = np.nonzero((x > self.threshold2))[0]

        self.left = TreeBoosterClassificationTernary(self.X, self.g, self.h, self.params,
                                self.max_depth - 1, self.idxs[left_idx])
        self.middle = TreeBoosterClassificationTernary(self.X, self.g, self.h, self.params,
                                  self.max_depth - 1, self.idxs[middle_idx])
        self.right = TreeBoosterClassificationTernary(self.X, self.g, self.h, self.params,
                                 self.max_depth - 1, self.idxs[right_idx])

    @property
    def is_leaf(self):
        return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X.values[self.idxs, feature_idx]
        g, h = self.g[self.idxs], self.h[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.
        sum_g_middle, sum_h_middle = 0., 0.

        for i in range(0, self.n - 2):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i; sum_g_right -= g_i; sum_g_middle += g_i
            sum_h_left += h_i; sum_h_right -= h_i; sum_h_middle += h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:
                continue
            if sum_h_right < self.min_child_weight:
                break

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                          + (sum_g_middle**2 / (sum_h_middle + self.reg_lambda))
                          + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                          - (sum_g**2 / (sum_h + self.reg_lambda))
                          ) - self.gamma

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold1 = (x_i + x_i_next) / 2
                self.threshold2 = (x_i_next + sort_x[i + 2]) / 2

    def predict(self, X):
        return np.array([self._predict_row(row) for i, row in X.iterrows()])

    def _predict_row(self, row):
        if self.is_leaf:
            return self.value
        if row[self.split_feature_idx] <= self.threshold1:
            child = self.left
        elif row[self.split_feature_idx] <= self.threshold2:
            child = self.middle
        else:
            child = self.right
        return child._predict_row(row)

    # def to_dot(self, feature_names):
    #     if self.is_leaf:
    #         return f'leaf ({self.value:.2f})'

    #     left_dot = self.left.to_dot(feature_names) if self.left else 'leaf (0.0)'
    #     middle_dot = self.middle.to_dot(feature_names) if self.middle else 'leaf (0.0)'
    #     right_dot = self.right.to_dot(feature_names) if self.right else 'leaf (0.0)'

    #     split_feature_name = feature_names[self.split_feature_idx]
    #     condition = f'{split_feature_name} <= {self.threshold1}'
    #     condition_middle = f'{self.threshold1} < {split_feature_name} <= {self.threshold2}'

    #     return f'node {{split: {split_feature_name}, thresholds: [{self.threshold1}, {self.threshold2}]\nleft: {left_dot}\nmiddle: {middle_dot}\nright: {right_dot}}}'

In [ ]:
def find_hyperparams(X_train, X_test, y_train, y_test):
    best_score = 100

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    learning_rates = [0.01, 0.05, 0.1]
    max_depths = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    reg_lambdas = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    gammas = [0.5, 0.75, 1.0, 1.5, 1.75, 2.0]
    min_child_weights = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

    combs = list(product(*[learning_rates, max_depths, reg_lambdas, gammas, min_child_weights]))

    for comb in tqdm(combs):
        learning_rate, max_depth, reg_lambda, gamma, min_child_weight = comb

        params = {
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'subsample': 1.0,
            'reg_lambda': reg_lambda,
            'gamma': gamma,
            'min_child_weight': min_child_weight,
            'base_score': 0.0,
            'device': 'cuda:0'
        }

        model_xgb = xgb.train(params, dtrain, num_boost_round=50)
        pred_xgb = model_xgb.predict(dtest)
        pred_xgb[pred_xgb < 0.5] = 0
        pred_xgb[pred_xgb >= 0.5] = 1

        score = LogLoss().loss(y_test, pred_xgb)

        if score < best_score:
            best_score = score
            best_params = params

    return best_params

In [ ]:
best_params = find_hyperparams(X_train, X_test, y_train, y_test)

  0%|          | 0/12600 [00:00<?, ?it/s]

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

model_xgb = xgb.train(best_params, dtrain, num_boost_round=50)
pred_xgb = model_xgb.predict(dtest)

print(f'XGBosst Classification best params: {best_params.items()}')

print(f'LogLoss error xgboost score: {LogLoss().loss(y_test, pred_xgb)}')

pred_xgb[pred_xgb < 0.5] = 0
pred_xgb[pred_xgb >= 0.5] = 1
get_metrics(y_test, pred_xgb)

XGBosst Classification best params: dict_items([('learning_rate', 0.05), ('max_depth', 5), ('subsample', 1.0), ('reg_lambda', 1.0), ('gamma', 0.5), ('min_child_weight', 15), ('base_score', 0.0), ('device', 'cuda:0')])
LogLoss error xgboost score: 0.49854762172489836
Accuracy: 0.956140350877193
Precision: 0.9726027397260274
Recall: 0.9594594594594594
F1 score: 0.9659863945578231


In [ ]:
best_params = {
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'reg_lambda': 1.0,
    'gamma': 0.5,
    'min_child_weight': 15,
    'base_score': 0.0,
    'method': 'exact'
}

In [ ]:
model_scratch_binary_split = XGBoostClassificationBinarySplit(best_params, random_seed=37)
model_scratch_binary_split.fit(X_train, y_train, LogLoss(), num_boost_round=50)

In [ ]:
model_scratch_ternary_split = XGBoostClassificationTernarySplit(best_params, random_seed=37)
model_scratch_ternary_split.fit(X_train, y_train, LogLoss(), num_boost_round=50)

In [ ]:
pred_scratch_binary = model_scratch_binary_split.predict(X_test)
pred_scratch_ternary = model_scratch_ternary_split.predict(X_test)

In [ ]:
print(f'LogLoss error binary scratch score: {LogLoss().loss(y_test, pred_scratch_binary)}')

pred_scratch_binary[pred_scratch_binary < 0.5] = 0
pred_scratch_binary[pred_scratch_binary >= 0.5] = 1
get_metrics(y_test, pred_scratch_binary)

LogLoss error binary scratch score: 0.5101767755777122
Accuracy: 0.9385964912280702
Precision: 0.958904109589041
Recall: 0.9459459459459459
F1 score: 0.9523809523809523


In [ ]:
print(f'Logloss error ternary scratch score: {LogLoss().loss(y_test, pred_scratch_ternary)}')

pred_scratch_ternary[pred_scratch_ternary < 0.5] = 0
pred_scratch_ternary[pred_scratch_ternary >= 0.5] = 1
get_metrics(y_test, pred_scratch_ternary)

Logloss error ternary scratch score: 0.5128101324019156
Accuracy: 0.9210526315789473
Precision: 0.9452054794520548
Recall: 0.9324324324324325
F1 score: 0.9387755102040816


In [149]:
best_params = {
    'learning_rate': 0.1,
    'max_depth': 15,
    'subsample': 1.0,
    'reg_lambda': 4.0,
    'gamma': 1.0,
    'min_child_weight': 20,
    'base_score': 0.0,
    'method': 'exact'
}

In [150]:
model_scratch_binary_split = XGBoostClassificationBinarySplit(best_params, random_seed=37)
model_scratch_binary_split.fit(X_train, y_train, LogLoss(), num_boost_round=150)

In [151]:
model_scratch_ternary_split = XGBoostClassificationTernarySplit(best_params, random_seed=37)
model_scratch_ternary_split.fit(X_train, y_train, LogLoss(), num_boost_round=150)

In [152]:
pred_scratch_binary = model_scratch_binary_split.predict(X_test)
pred_scratch_ternary = model_scratch_ternary_split.predict(X_test)

In [153]:
print(f'LogLoss error binary scratch score: {LogLoss().loss(y_test, pred_scratch_binary)}')

pred_scratch_binary[pred_scratch_binary < 0.5] = 0
pred_scratch_binary[pred_scratch_binary >= 0.5] = 1
get_metrics(y_test, pred_scratch_binary)

LogLoss error binary scratch score: 0.5109944219690449
Accuracy: 0.9210526315789473
Precision: 0.9452054794520548
Recall: 0.9324324324324325
F1 score: 0.9387755102040816


In [154]:
print(f'Logloss error ternary scratch score: {LogLoss().loss(y_test, pred_scratch_ternary)}')

pred_scratch_ternary[pred_scratch_ternary < 0.5] = 0
pred_scratch_ternary[pred_scratch_ternary >= 0.5] = 1
get_metrics(y_test, pred_scratch_ternary)

Logloss error ternary scratch score: 0.5102743164698879
Accuracy: 0.9210526315789473
Precision: 0.9452054794520548
Recall: 0.9324324324324325
F1 score: 0.9387755102040816
